In [ ]:
'''
https://en.wikipedia.org/wiki/Assignment_problem
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html

order of lists is important (eg, student 1 before student 2). in case of draw the preferences of the first list are preferred
student1    [3, 2, 1] -> student 1 prefers topic 3, then 2 and lastly the first topic (C > B > A)

example 1:
weight_of topic1 topic2 topic3
student1    [3, 2, 1], 
student2    [1, 3, 2], 
student3    [2, 1, 3]
->
adjusted (plus one) array indices of best solution: [3 1 2]
S1 gets topic3, S2 gets topic 1, S3 gets topic 2

example 2:
not an issue, if more topics than students
[4,3,1,2], 
[2,4,1,3], 
[2,1,3,4], 
->
adjusted (plus one) array indices of best solution: [3 1 2]

example 3:
if less topics than students, and each topic is assigned more than once:
list each topic accordingly to the number it is assigned. eg, topic1 can be chosen 4 times and topic2 3 times
[1,1,1,1,2,2,2]
[2,2,2,2,1,1,1]
'''

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment

df = pd.read_excel(r"D:\Dokumente\WMA\Veranstaltungen\4 WissArbeiten\W21\Übersicht_Teilnehmer_Master-Thesis.xlsx", sheet_name="Tabelle1")

df = df[df["Themen-Präferenzen"].str.contains("zurückgetreten")==False] # drop zurückgetreten students
df["Themen-Präferenzen"] = df["Themen-Präferenzen"].str.replace(' ', '')
df["Themen-Präferenzen"] = df["Themen-Präferenzen"].apply(eval) # lists are stored as strings in pandas -> convert with apply(eval)
df

In [5]:
# import from xlxs file. ideally lists are sorted by grades. output to same xlsx

# preferences = [
#     [4,3,1,2], 
#     [2,4,1,3], 
#     [2,1,3,4], 
#     ]

preferences = df['Themen-Präferenzen'].to_list()

cost = np.array(preferences)
row_ind, col_ind = linear_sum_assignment(cost)
adj_ind = [i+1 for i in col_ind]

print(f"Available Topics: {len(preferences[0])}")
print(f"Assigned Topics: {len(col_ind)}")
print(f"Number of students: {df.index[-1] + 1}")
print(f"array indices of best solution: {col_ind}")
print(f"adjusted (plus one) indices of best solution: {adj_ind}\n")


# result = [f"Student {i}: Topic {v+1}" for i,v in enumerate(col_ind, start=1)]
# for i in result:
#     print(i)

for i,v in enumerate(preferences):
    selected_topic = col_ind[i]+1
    pref_of_selected_topic = v[col_ind[i]]
    print(f"Student {i+1}: Assigned topic {selected_topic} (selected as #{pref_of_selected_topic})")
print(f"\nTotal Sum of preferences (lowest sum possible): {cost[row_ind, col_ind].sum()}\n")

Available Topics: 12
Assigned Topics: 10
Number of students: 10
array indices of best solution: [5 0 8 2 1 3 4 6 9 7]
adjusted (plus one) indices of best solution: [6, 1, 9, 3, 2, 4, 5, 7, 10, 8]

Student 1: Assigned topic 6 (selected as #2)
Student 2: Assigned topic 1 (selected as #2)
Student 3: Assigned topic 9 (selected as #1)
Student 4: Assigned topic 3 (selected as #1)
Student 5: Assigned topic 2 (selected as #2)
Student 6: Assigned topic 4 (selected as #1)
Student 7: Assigned topic 5 (selected as #1)
Student 8: Assigned topic 7 (selected as #1)
Student 9: Assigned topic 10 (selected as #4)
Student 10: Assigned topic 8 (selected as #5)

Total Sum of preferences (lowest sum possible): 20



In [ ]:
df["scipy.optimize.linear_sum_assignment"] = adj_ind
df

In [9]:
df2 = pd.read_excel(r"D:\Dokumente\WMA\Veranstaltungen\4 WissArbeiten\W21\Übersicht_Teilnehmer_Master-Thesis.xlsx", sheet_name="Themen")
df2

df3 = df1.merge(df2, on="Thema")


,Thema,Betreuer,Titel DE
0,1,AW,Der Einfluss einer offenen Fehlerkultur auf Es...
1,2,AW,Messung der Managementrationalität
2,3,AW,Konzeption eines Debiasing-Workshops für das P...
3,4,MG,Kognitive Verzerrungen am Heimarbeitsplatz (Ho...
4,5,MG,Wie Innovationskultur in Unternehmen das Entsc...
5,6,HG,Wie selektieren externe Direktoren ihre Vorsta...
6,7,HG,Wie groß ist der Einfluss externer Direktoren ...
7,8,HG,Beeinflusst Kostenremanenz die Mitarbeiterzufr...
8,9,HG,Management Earnings Guidance vs. Machine Learn...
9,10,RS,Der Einfluss von Failure Awards auf Creativity...
